### Задание 2. SQL
#### 2.1 Очень усердные ученики.

**Условие**
Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".
Назовём очень усердным учеником того пользователя, который за текущий месяц правильно решил 20 задач.  
  
Дана таблица peas:

Название атрибута|	Тип атрибута |	Смысловое значение
-------|---------|-----------------------------
st_id  |int      |ID ученика 
       |         |
timest |timestamp|Время решения карточки
       |         |
correct|bool     |Правильно ли решена горошина? 
       |         |
subject|text     |Дисциплина, в которой находится горошина


Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов за март 2020 года.

NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

**Генерируем данные**

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
st_id = np.random.randint(1000, 6000, 1000)

In [3]:
st_id = st_id.tolist() * 5

In [4]:
timest = pd.date_range(start="2020-01-01", end="2020-12-31").strftime("%Y-%m-%d").tolist()
timest = timest * 14
random.shuffle(timest)
timest = timest[0:5000]

In [5]:
correct = np.random.randint(0, 2, 5000)

In [6]:
subject = ['math'] * 2500
subject.extend(['physics'] * 1000)
subject.extend(['chemistry'] * 800)
subject.extend(['biology'] * 700)
random.shuffle(subject)

In [7]:
# Initialize data to Dicts of series.
d = {'st_id': pd.Series(st_id),
     'timest': timest,
     'correct': pd.Series(correct),
     'subject': pd.Series(subject)}
 
# creates Dataframe.
df = pd.DataFrame(d)

**Выгружаем данные для обработки в postgreSQL**

In [8]:
df.to_csv('peas.csv', sep=',')

**Запрос будет выглядеть так:**


    select count(distinct st_id)  
    from (  
		select st_id, 
                date_trunc('month',timest) as month, 
                sum(correct::int) as tasks  
		from peas  
		where date_trunc('month',timest) = '2020-03-01'  
		group by st_id, date_trunc('month',timest)  
		having sum(correct::int) > 19  
        ) a

#### 2.2 Оптимизация воронки

**Условие**
Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Дана таблицы: peas (см. выше), studs:

Название атрибута|	Тип атрибута| 	Смысловое значение
-----------------|--------------|---------------------
st_id|	int| 	ID ученика|
test_grp|	text| 	Метка ученика в данном эксперименте|

и checks:

Название атрибута|	Тип атрибута| 	Смысловое значение
-----------------|--------------|---------------------
st_id|	int |	ID ученика|
sale_time|	timestamp|	Время покупки|
money|	int|	Цена, по которой приобрели данный курс|
subject|	text| 	Дисциплина, на которую приобрели полный доступ|

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:  

- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

**Сгенерируем данные для таблицы studs**  
Кроме st_id студентов из табл. peas там должны быть st_id и других студентов, регистрировавшихся на платформе.	

In [9]:
st_id_all = df.st_id.to_list()
st_id_add = np.random.randint(6000, 21000, 10000).tolist()
st_id_all.extend(st_id_add)
st_id_all = list(set(st_id_all))

In [11]:
test_grp = ['A'] * 3000
test_grp.extend(['B'] * 5192)
random.shuffle(test_grp)

In [13]:
# Initialize data to Dicts of series.
f = {'st_id': pd.Series(st_id_all),
     'test_grp': pd.Series(test_grp)}
 
# creates Dataframe.
studs = pd.DataFrame(f)

In [14]:
studs.to_csv('studs.csv', sep=',')

**Сгенерируем данные для таблицы checks** 

In [15]:
st_id_pay = df.st_id.to_list()

In [16]:
st_id_pay = list(set(st_id_pay))[100:500]

In [17]:
subject_checks = ['math'] * 200
subject_checks.extend(['physics'] * 100)
subject_checks.extend(['chemistry'] * 70)
subject_checks.extend(['biology'] * 30)
random.shuffle(subject_checks)

In [19]:
# Initialize data to Dicts of series.
h = {'st_id': pd.Series(st_id_pay),
     'subject': pd.Series(subject_checks)}
 
# creates Dataframe.
checks = pd.DataFrame(h)

In [20]:
checks['money'] = checks.subject.apply(
    lambda x: 20000 if x == 'math' 
    else 15000 if x == 'physics'  
    else 10000 if x == 'chemistry' 
    else 5000)

In [21]:
checks.head()

,st_id,subject,money
0,4360,math,20000
1,4361,physics,15000
2,4363,math,20000
3,2316,math,20000
4,4365,physics,15000


**Выгружаем данные для обработки в postgreSQL**

In [22]:
checks.to_csv('checks.csv', sep=',')

**Запрос будет выглядеть так:**

    select test_grp,
		sum(c.money)/count(s.st_id) as ARPU,
		sum(c.money)/(select count(distinct st_id)
						from peas p ) as ARPAU,
		count(distinct c.st_id)::float/count(s.st_id) as CR_in_purch,
		count(distinct c.st_id)::float/(select count(distinct st_id)
						from peas p ) as CR_act_in_purch,
		SUM(CASE WHEN c.subject = 'math' THEN 1 ELSE 0 END)::float/ (select count(distinct st_id)
																from peas p
																where subject = 'math') as CR_act_math_in_purch
    from studs s
    left join checks c using(st_id)
    GROUP BY
        test_grp